In [1]:
from copy import copy

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

from torch.autograd import Variable

# Load data

In [2]:
txt = ''

In [3]:
with open('data/one_txt/sanitized_blogger.txt') as f:
    txt += f.read()

len(txt)

442724

In [4]:
with open('data/one_txt/sanitized_wordpress.txt') as f:
    txt += f.read()

len(txt)

3216695

In [5]:
vocab = sorted(list(set(txt)))
n_vocab = len(vocab)
print(''.join(vocab))

 !"$%'()+,-./0123456789:;=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz~°àâçèéêëîïôùûœо€


In [6]:
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for idx, char in enumerate(vocab)}

In [7]:
train_frac = 3. / 4
train_txt = txt[:int(len(txt) * train_frac)]
test_txt = txt[int(len(txt) * train_frac):]

# Fixed-size RNN

This is a model which operates on a **fixed** amount of input characters (`n_chars`), and attempts to predict the character that comes after them.

The hidden state is reset for each new sequence of `n_chars` characters (*stateless*).

In [8]:
n_chars = 3

In [9]:
def get_n_sized_chunks(s, n):
    """
    Yield successive n-sized chunks from a string.
    Discard the last chunk if not of size n.
    """
    for i in range(0, len(s), n):
        chunk = s[i:i + n]
        if len(chunk) == n:
            yield chunk

In [10]:
def get_data_tensor(txt, n_chars):
    chunks = list(get_n_sized_chunks(txt, n=n_chars))
    data_tensor = torch.tensor([[char_to_idx[char] for char in chunk] for chunk in chunks][:-1])
    return data_tensor

In [11]:
def get_labels_tensor(txt, n_chars):
    chars = txt[n_chars::n_chars][:len(txt) // n_chars - 1]
    labels_tensor = torch.tensor([char_to_idx[char] for char in chars])
    return labels_tensor

In [12]:
train_data_tensor = get_data_tensor(train_txt, n_chars)
print(train_data_tensor.size())

train_labels_tensor = get_labels_tensor(train_txt, n_chars)
print(train_labels_tensor.size())

torch.Size([804172, 3])
torch.Size([804172])


In [13]:
train_ds = TensorDataset(train_data_tensor, train_labels_tensor)
train_dl = DataLoader(train_ds, batch_size=1024)

In [14]:
test_data_tensor = get_data_tensor(test_txt, n_chars)
print(test_data_tensor.size())

test_labels_tensor = get_labels_tensor(test_txt, n_chars)
print(test_labels_tensor.size())

torch.Size([268057, 3])
torch.Size([268057])


In [15]:
test_ds = TensorDataset(test_data_tensor, test_labels_tensor)
test_dl = DataLoader(test_ds, batch_size=1024)

In [16]:
def generate1(model, s, n, kind):

    assert kind in ('top', 'multinomial')
    assert len(s) == n_chars

    final_s = s

    for _ in range(n):

        chars = get_data_tensor(s + '   ', n_chars)
        preds = model(chars)

        if kind == 'top':
            pred_idx = preds.argmax().item()

        elif kind == 'multinomial':
            pred_idx = torch.multinomial(preds.exp(), 1).item()
            
        pred_char = idx_to_char[pred_idx]
        s = s[1:] + pred_char
        final_s += pred_char

    return final_s

![](img/rnn1.jpg)

In [17]:
class FixedSizeRNN(nn.Module):
    def __init__(self, n_vocab, n_factors, n_hidden, n_chars):
        super().__init__()
        self.n_chars = n_chars
        self.e = nn.Embedding(n_vocab, n_factors)
        self.input_weights = nn.Linear(n_factors, n_hidden)
        self.hidden_weights = nn.Linear(n_hidden, n_hidden)
        self.output_weights = nn.Linear(n_hidden, n_vocab)

    def forward(self, chars):

        hidden = torch.zeros([len(chars), n_hidden])

        for i in range(self.n_chars):
            input = F.relu(self.input_weights(self.e(chars[:, i])))
            hidden = torch.tanh(self.hidden_weights(input + hidden))

        output = F.log_softmax(self.output_weights(hidden), dim=1)
        
        return output

In [18]:
n_fac = n_vocab // 2
n_hidden = 100

In [19]:
model1 = FixedSizeRNN(n_vocab, n_fac, n_hidden, n_chars)

In [20]:
optimizer1 = torch.optim.Adam(model1.parameters(), 1e-2)
criterion1 = nn.NLLLoss()

In [21]:
%%time

epochs = 50

for epoch in range(1, epochs + 1):
    
    print(f'epoch: {epoch}')
    
    train_loss_sum, train_batches_nb = 0, 0
    for i, (data, labels) in enumerate(train_dl, 1):
        output = model1(data)
        optimizer1.zero_grad()
        loss = criterion1(output, labels)
        train_loss_sum, train_batches_nb = train_loss_sum + loss.item(), train_batches_nb + 1
        loss.backward()
        optimizer1.step()

    test_loss_sum, test_batches_nb = 0, 0
    for data, labels in test_dl:
        loss = criterion1(model1(data), labels)
        test_loss_sum, test_batches_nb = test_loss_sum + loss.item(), test_batches_nb + 1

    if epoch == 1 or epoch % 10 == 0 or epoch == epochs:

        print()
        
        print(f'train loss: {round(train_loss_sum / train_batches_nb, 2)}')
        print(f'test loss: {round(test_loss_sum / test_batches_nb, 2)}')
        
        print()
        
        for kind in ('top', 'multinomial'):
            print(f'sample {kind}: ' + generate1(model1, 'je ', 200, kind))
            print()

epoch: 1

train loss: 2.05
test loss: 1.96

sample top: je se de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la res de la r

sample multinomial: je la d'avant comm sant c'estion rapaux a malaguitilitand modisent très loit commext. Lorjatitans d'Armarme d'ouche d'alles tera la mune mos pourn mous des solie d'une Tivien de vie; la mamit une n'avie 

epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10

train loss: 1.83
test loss: 1.84

sample top: je son de la route de la route de la route de la route de la route de la route de la route de la route de la route de la route de la route de la route de la route de la route de la route de la route de l

sample multinomial: je notramemboulmondicé du rop ma où la tôt je sais avons l'attain a salais et gôtes du p payss à vous que, visage, voie semag s'adi à, sonther du du petiture sa

# Variable-size model

This is a model which operates on a **variable** amount of input characters, and attempts to predict the next character **after each input character**.

In [22]:
def get_data(txt, bs):
    """
    Split `txt` into `bs` chunks.

    Each chunk has size `n`, `n` being as big as possible.
    Chunks are organized as columns in the result, making the final size `n * bs`.
    """

    txt = [char_to_idx[c] for c in txt]
    
    # Shrink `len(txt)` to a multiple of `bs`
    txt_len = (len(txt) // bs) * bs
    txt = txt[:txt_len]

    # Cut `txt` into `bs` distinct chunks
    data = torch.tensor(txt).view(bs, -1)
    data = data.transpose(0, 1).contiguous()
    
    return data

In [23]:
def get_batches(data, bptt):
    """
    Yield `(data_batch, labels_batch)` batches from `data`.

    At each iteration, the two batches have the same `bptt * bs` size,
    except for the last batch which may have less than `bptt` rows.

    `data_batch` contains `bptt`-sized chunks of `data`.
    `labels_batch` contains `bptt`-sized chunks of `data`, offseted by 1.
    """

    # Cut `data` into two 2-dimensional chunks of size `bptt * bs`.
    # Last chunk may be less than `bptt` rows.
    while len(data) != 0:

        # Take (at most) bptt rows with offset 1 for labels
        labels_batch = data[1:bptt+1, :]
        # Take bptt rows as the labels with offset 0 for train
        data_batch = data[:len(labels_batch), :]

        if len(labels_batch) > 0:
            yield data_batch, labels_batch

        # Move on to next train train/labels rows
        data = data[bptt:]

In [24]:
i = 1
data = get_data(train_txt, bs=3)
for data_batch, labels_batch in get_batches(data, bptt=5):
    
    print(f'data:')
    print(data_batch)

    print(f'labels:')
    print(labels_batch)

    print()
    print()
    
    i += 1
    if i > 2:
        break

data:
tensor([[42, 59, 62],
        [60, 67, 55],
        [60, 67, 68],
        [63, 59, 61],
        [57, 68, 87]])
labels:
tensor([[60, 67, 55],
        [60, 67, 68],
        [63, 59, 61],
        [57, 68, 87],
        [63, 59, 73]])


data:
tensor([[63, 59, 73],
        [59, 72,  0],
        [75,  0, 55],
        [73, 70, 76],
        [59, 66, 59]])
labels:
tensor([[59, 72,  0],
        [75,  0, 55],
        [73, 70, 76],
        [59, 66, 59],
        [67, 75, 57]])




In [25]:
def generate2(model, s, n, kind):

    assert kind in ('top', 'multinomial')

    model.reset(1)

    res = s
    for _ in range(n):
        data = get_data(s, 1)
        preds = model(data)[-1]

        if kind == 'top':
            pred_idx = preds.argmax().item()

        elif kind == 'multinomial':
            pred_idx = torch.multinomial(preds.exp(), 1).item()

        pred_char = idx_to_char[pred_idx]
        res += pred_char
        s = s[1:] + pred_char
        
    return res

![](img/rnn2.jpg)

In [26]:
class VariableLengthRNN(nn.Module):
    def __init__(self, n_vocab, n_fac, n_hidden, kind):
        super().__init__()
        
        assert kind in ('stateless', 'stateful')
        self.kind = kind
        
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.e = nn.Embedding(n_vocab, n_fac)
        self.output_weights = nn.Linear(n_hidden, n_vocab)

    def forward(self, data):
        input = self.e(data)
        output, h = self.rnn(input, self.hidden_weights)
        
        if self.kind == 'stateful':
            # Keep the hidden state between each minibatch
            self.hidden_weights = Variable(h.data)
        
        output = self.output_weights(output)
        output = F.log_softmax(output, dim=-1)
        return output

    def reset(self, bs):
        self.hidden_weights = torch.zeros([1, bs, n_hidden])

## Stateless RNN

The hidden state is thown away from one mini-batch to another.

In [27]:
n_fac = n_vocab // 2
n_hidden = 256
bs = 1024
bptt = 70

In [28]:
model2 = VariableLengthRNN(n_vocab, n_fac, n_hidden, 'stateless')

In [29]:
def nll_loss_seq(output, labels):
    _, _, n_vocab = output.size()
    output = output.view(-1, n_vocab)
    labels = labels.reshape(-1)
    return F.nll_loss(output, labels)

In [30]:
optimizer2 = torch.optim.Adam(model2.parameters(), 1e-2)
criterion2 = nll_loss_seq

In [31]:
train_data = get_data(train_txt, bs)
test_data = get_data(test_txt, bs)

In [32]:
%%time

epochs = 300

for epoch in range(1, epochs + 1):
    
    print(f'epoch: {epoch}')

    model2.reset(bs)

    train_loss_sum, train_batches_nb = 0, 0
    for i, (data, labels) in enumerate(get_batches(train_data, bptt), 1):
        output = model2(data)
        optimizer2.zero_grad()
        loss = criterion2(output, labels)
        train_loss_sum, train_batches_nb = train_loss_sum + loss.item(), train_batches_nb + 1
        loss.backward()
        optimizer2.step()

    test_loss_sum, test_batches_nb = 0, 0
    for data, labels in get_batches(test_data, bptt):
        loss = criterion2(model2(data), labels)
        test_loss_sum, test_batches_nb = test_loss_sum + loss.item(), test_batches_nb + 1

    if epoch == 1 or epoch % 10 == 0 or epoch == epochs:

        print()
        
        print(f'train loss: {round(train_loss_sum / train_batches_nb, 2)}')
        print(f'test loss: {round(test_loss_sum / test_batches_nb, 2)}')
        
        print()
        
        for kind in ('top', 'multinomial'):
            print(f'sample {kind}: ' + generate2(model2, 'je ', 200, kind))
            print()

epoch: 1

train loss: 2.74
test loss: 2.28

sample top: je de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de

sample multinomial: je limtrmint bsui Me l ymovheitéasélens lle.conh ilompi0ongle ple deris rtagss chausont si G. datsonspentspélil teurpourelmv le pe re 0articuidient me/nç pamest ces Cotruet ène c'eitrorpinfsiirs.. es pui

epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10

train loss: 1.53
test loss: 1.54

sample top: je nous de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de 

sample multinomial: je rence ves débus jouexccarentre a prés exclus reperdroires 54 3 jondeune resons jotiont de Payante... Langle qui de quel mummetterre et de beau Plis son serve

epoch: 142
epoch: 143
epoch: 144
epoch: 145
epoch: 146
epoch: 147
epoch: 148
epoch: 149
epoch: 150

train loss: 1.37
test loss: 1.41

sample top: je partien prochangers de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le 

sample multinomial: je rentre des qui raphiter uti vu 17. Kum de se yao les. Le seron intre est temiersementôt au on se par n'est.  l'ange demier en son cons legien Un che attrus apractière je est m'espitésie complerexi se 

epoch: 151
epoch: 152
epoch: 153
epoch: 154
epoch: 155
epoch: 156
epoch: 157
epoch: 158
epoch: 159
epoch: 160

train loss: 1.35
test loss: 1.42

sample top: je rence de la réciation au de la réciation au de la réciation au de la réciation au de la réciation au de la réciation au de la réciation au de la réciation au de la réciation au de la réciation au de l

sample multinomial: je nous et ou. J'ailes poul dans portesse 

epoch: 282
epoch: 283
epoch: 284
epoch: 285
epoch: 286
epoch: 287
epoch: 288
epoch: 289
epoch: 290

train loss: 1.49
test loss: 1.53

sample top: je sont de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de 

sample multinomial: je du plusuryp momident acces. Ouiale stymentant : de bile du elle !). Cesina répartour militaireurez la cet à que qu'ile (à 1500 déjeux pomment : Je ver de rouver sont surppres désol, margé de n'y le mo

epoch: 291
epoch: 292
epoch: 293
epoch: 294
epoch: 295
epoch: 296
epoch: 297
epoch: 298
epoch: 299
epoch: 300

train loss: 1.48
test loss: 1.51

sample top: je sons de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de 

sample multinomial: je au toutes écoles Irieuse dépermachautur

epoch: 422
epoch: 423
epoch: 424
epoch: 425
epoch: 426
epoch: 427
epoch: 428
epoch: 429
epoch: 430

train loss: 1.72
test loss: 1.73

sample top: je partit de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le d

sample multinomial: je étainion En des aus élailleigs miants je à hublacue toutes douje faurrivemblopoliende nouverdinans dementés du segment pas d'undion monique un peus, j'avoier il ou des peur le ches très. Nouve les peu

epoch: 431
epoch: 432
epoch: 433
epoch: 434
epoch: 435
epoch: 436
epoch: 437
epoch: 438
epoch: 439
epoch: 440

train loss: 1.66
test loss: 1.68

sample top: je par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par 

sample multinomial: je marges millectickéque le poues a un la 

epoch: 562
epoch: 563
epoch: 564
epoch: 565
epoch: 566
epoch: 567
epoch: 568
epoch: 569
epoch: 570

train loss: 1.59
test loss: 1.6

sample top: je de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de

sample multinomial: je pieux prachest de les a à une le beaux Hette forestiallagera la n'est on  aussi en fracsppéshini quille-Zéllagérer offre autreude tra sud mie. Le plant mus interrivalentièrestiquelle, bon au Euxe du d

epoch: 571
epoch: 572
epoch: 573
epoch: 574
epoch: 575
epoch: 576
epoch: 577
epoch: 578
epoch: 579
epoch: 580

train loss: 1.52
test loss: 1.56

sample top: je de la par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par par pa

sample multinomial: je ce notos-notos petit-deming-ils parler d

epoch: 702
epoch: 703
epoch: 704
epoch: 705
epoch: 706
epoch: 707
epoch: 708
epoch: 709
epoch: 710

train loss: 1.57
test loss: 1.65

sample top: je maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine pas de maine

sample multinomial: je blade. Langetentiperaine 1ergigne. Pardans mancil, que ros. Ils façonse efin. L'utitnevians peu. Juent les a la visitudieussiant le et expémis de me voyagore " : " pours. Quas un jours meinuitemps tas

epoch: 711
epoch: 712
epoch: 713
epoch: 714
epoch: 715
epoch: 716
epoch: 717
epoch: 718
epoch: 719
epoch: 720

train loss: 1.56
test loss: 1.59

sample top: je pas de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de le de l

sample multinomial: je sontonidiné, trocuait suraine grajonner

epoch: 842
epoch: 843
epoch: 844
epoch: 845
epoch: 846
epoch: 847
epoch: 848
epoch: 849
epoch: 850

train loss: 1.57
test loss: 1.6

sample top: je sons pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me pas de la me 

sample multinomial: je et plats de touc dants déparhent il fait de fréale (les je sur mon qui. Un moillommence qu'ils de " est un égan d'a perméen à qui pande pour, pais à contrialent lors puis quit l'en  ma rou artonistoch

epoch: 851
epoch: 852
epoch: 853
epoch: 854
epoch: 855
epoch: 856
epoch: 857
epoch: 858
epoch: 859
epoch: 860

train loss: 1.55
test loss: 1.59

sample top: je sons par le de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mon de la mo

sample multinomial: je et mant esproc puis, véleisse qu'on du c

KeyboardInterrupt: 

## Stateful RNN

The hidden state is be memorized from one mini-batch to another (hence *stateful*), but reset between epochs, and at predict time.

In [33]:
n_fac = n_vocab // 2
n_hidden = 256
bs = 1024
bptt = 70

In [34]:
model3 = VariableLengthRNN(n_vocab, n_fac, n_hidden, 'stateful')

In [35]:
def nll_loss_seq(output, labels):
    _, _, n_vocab = output.size()
    output = output.view(-1, n_vocab)
    labels = labels.reshape(-1)
    return F.nll_loss(output, labels)

In [36]:
optimizer3 = torch.optim.Adam(model3.parameters(), 1e-2)
criterion3 = nll_loss_seq

In [37]:
train_data = get_data(train_txt, bs)
test_data = get_data(test_txt, bs)

KeyboardInterrupt: 

In [ ]:
%%time

epochs = 2000

for epoch in range(1, epochs + 1):
    
    print(f'epoch: {epoch}')

    model3.reset(bs)

    train_loss_sum, train_batches_nb = 0, 0
    for i, (data, labels) in enumerate(get_batches(train_data, bptt), 1):
        output = model3(data)
        optimizer3.zero_grad()
        loss = criterion3(output, labels)
        train_loss_sum, train_batches_nb = train_loss_sum + loss.item(), train_batches_nb + 1
        loss.backward()
        optimizer3.step()

    test_loss_sum, test_batches_nb = 0, 0
    for data, labels in get_batches(test_data, bptt):
        loss = criterion3(model3(data), labels)
        test_loss_sum, test_batches_nb = test_loss_sum + loss.item(), test_batches_nb + 1

    if epoch == 1 or epoch % 10 == 0 or epoch == epochs:

        print()
        
        print(f'train loss: {round(train_loss_sum / train_batches_nb, 2)}')
        print(f'test loss: {round(test_loss_sum / test_batches_nb, 2)}')
        
        print()
        
        for kind in ('top', 'multinomial'):
            print(f'sample {kind}: ' + generate2(model3, 'je ', 200, kind))
            print()